In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler , RobustScaler 

In [2]:
#  수명주기 -> 숫자로 인코딩
# 직급명 , 회사명제거, 거래소코드 , 상장일 , 대분류 삭제컬럼

# 회계년도 기준으로 train , test split 
#                   1) 2013~2019  ,  2020~2021 : 년도 기준으로나눈거고

In [3]:
df = pd.read_csv('datasets\이상치제거후vif공선성제거\윈저_vif.csv',index_col=0)
df.columns

Index(['target', '거래소코드', '상장일', '수명주기', '통계청 한국표준산업분류 10차(대분류)', '회계년도',
       '회사명', '대주주지분변화분', '외국인지분분변화', '자산', '당기순이익(손실)', '비유동자산증가율', '유동자산증가율',
       '자기자본증가율', '매출액증가율', '정상영업이익증가율', '순이익증가율', '총포괄이익증가율', '매출액순이익률',
       '자기자본순이익률', '광고선전비대매출액비율', '비유동비율', '당좌비율', '현금비율', '부채비율',
       '이자보상배율(이자비용)', '유보액대납입자본배율', '투자집중도', '1인년간평균급여(직원)', '토빈Q', 'WW지수',
       'RDS', '산업평균_총자산증가율차분', '보통주식비율', '외국인_주식분포비율', '업력'],
      dtype='object')

In [4]:
df.target.value_counts()

0    6877
1    3030
Name: target, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9907 entries, 0 to 9906
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   target                 9907 non-null   int64  
 1   거래소코드                  9907 non-null   int64  
 2   상장일                    9907 non-null   object 
 3   수명주기                   9907 non-null   object 
 4   통계청 한국표준산업분류 10차(대분류)  9907 non-null   object 
 5   회계년도                   9907 non-null   int64  
 6   회사명                    9907 non-null   object 
 7   대주주지분변화분               9907 non-null   float64
 8   외국인지분분변화               9907 non-null   float64
 9   자산                     9907 non-null   float64
 10  당기순이익(손실)              9907 non-null   float64
 11  비유동자산증가율               9907 non-null   float64
 12  유동자산증가율                9907 non-null   float64
 13  자기자본증가율                9907 non-null   float64
 14  매출액증가율                 9907 non-null   float64
 15  정상영업

In [8]:
## 11~12년도는 y taget 설정조건으로 인하여 제거

bef_df = df[~(df['회계년도'].isin([2011,2012]))].reset_index(drop=True)

In [9]:
id_col = bef_df[['회사명','회계년도','상장일','통계청 한국표준산업분류 10차(대분류)','거래소코드']]
bef_df.drop(['회사명','거래소코드','상장일','통계청 한국표준산업분류 10차(대분류)'],axis=1,inplace=True)

In [10]:
### 성장주기 변수 onehot인코딩 get_dummy()
dummy = pd.get_dummies(bef_df['수명주기'])
bef_df.drop(['수명주기'],axis=1,inplace=True)
bef_df = pd.concat([bef_df,dummy],axis=1)

In [11]:
bef_df.columns

Index(['target', '회계년도', '대주주지분변화분', '외국인지분분변화', '자산', '당기순이익(손실)', '비유동자산증가율',
       '유동자산증가율', '자기자본증가율', '매출액증가율', '정상영업이익증가율', '순이익증가율', '총포괄이익증가율',
       '매출액순이익률', '자기자본순이익률', '광고선전비대매출액비율', '비유동비율', '당좌비율', '현금비율', '부채비율',
       '이자보상배율(이자비용)', '유보액대납입자본배율', '투자집중도', '1인년간평균급여(직원)', '토빈Q', 'WW지수',
       'RDS', '산업평균_총자산증가율차분', '보통주식비율', '외국인_주식분포비율', '업력', '도입기', '성숙기',
       '성장기', '쇠퇴기'],
      dtype='object')

In [12]:
### 회계년도 기준 split

train = bef_df[bef_df['회계년도']<=2019].reset_index(drop=True)
test = bef_df[bef_df['회계년도']>=2020].reset_index(drop=True)


train_y = train[['target']]
train_x = train[[ '대주주지분변화분', '외국인지분분변화', '자산', '당기순이익(손실)', '비유동자산증가율',
       '유동자산증가율', '자기자본증가율', '매출액증가율', '정상영업이익증가율', '순이익증가율', '총포괄이익증가율',
       '매출액순이익률', '자기자본순이익률', '광고선전비대매출액비율', '비유동비율', '당좌비율', '현금비율', '부채비율',
       '이자보상배율(이자비용)', '유보액대납입자본배율', '투자집중도', '1인년간평균급여(직원)', '토빈Q', 'WW지수',
       'RDS', '산업평균_총자산증가율차분', '보통주식비율', '외국인_주식분포비율', '업력', '도입기', '성숙기',
       '성장기', '쇠퇴기']]






test_y = test[['target']]
test_x = test[[ '대주주지분변화분', '외국인지분분변화', '자산', '당기순이익(손실)', '비유동자산증가율',
       '유동자산증가율', '자기자본증가율', '매출액증가율', '정상영업이익증가율', '순이익증가율', '총포괄이익증가율',
       '매출액순이익률', '자기자본순이익률', '광고선전비대매출액비율', '비유동비율', '당좌비율', '현금비율', '부채비율',
       '이자보상배율(이자비용)', '유보액대납입자본배율', '투자집중도', '1인년간평균급여(직원)', '토빈Q', 'WW지수',
       'RDS', '산업평균_총자산증가율차분', '보통주식비율', '외국인_주식분포비율', '업력', '도입기', '성숙기',
       '성장기', '쇠퇴기']]

scale_list = [ '대주주지분변화분', '외국인지분분변화', '자산', '당기순이익(손실)', '비유동자산증가율',
       '유동자산증가율', '자기자본증가율', '매출액증가율', '정상영업이익증가율', '순이익증가율', '총포괄이익증가율',
       '매출액순이익률', '자기자본순이익률', '광고선전비대매출액비율', '비유동비율', '당좌비율', '현금비율', '부채비율',
       '이자보상배율(이자비용)', '유보액대납입자본배율', '투자집중도', '1인년간평균급여(직원)', '토빈Q', 'WW지수',
       'RDS', '산업평균_총자산증가율차분', '보통주식비율', '외국인_주식분포비율', '업력']

## 범주형변수 잠시 분리

In [13]:
train_수명주기 = train_x[[ '도입기','성숙기', '성장기', '쇠퇴기']] 
test_수명주기 = test_x[[ '도입기','성숙기', '성장기', '쇠퇴기']]
# id_col
# 3개의 데이터프레임을 마지막에 다시 합쳐준다

# 윈저

## Standarscaler

In [14]:
scaler = StandardScaler()
### train
standar_train = scaler.fit_transform(train_x[scale_list])

임시_df= pd.DataFrame(standar_train, columns=scale_list)
standar_train_x = pd.concat([임시_df,train_수명주기],axis=1)

### test
standar_test = scaler.transform(test_x[scale_list])

임시_df= pd.DataFrame(standar_test, columns=scale_list)
standar_test_x = pd.concat([임시_df,test_수명주기],axis=1)


In [15]:
stand_train = pd.concat([standar_train_x, train_y],axis=1)
stand_test = pd.concat([standar_test_x, test_y],axis=1)

standar_df = pd.concat([stand_train,stand_test],axis=0).reset_index(drop=True)
standar_df = pd.concat([id_col,standar_df],axis=1)
standar_df.to_csv('./datasets/스케일링완료/윈저_standar.csv',encoding='utf-8-sig')

## 다시 csv로 저장

## MINMAXscaler

In [16]:
scaler = MinMaxScaler()
### train
minmax_train = scaler.fit_transform(train_x[scale_list])

임시_df= pd.DataFrame(minmax_train, columns=scale_list)
minmax_train_x = pd.concat([임시_df,train_수명주기],axis=1)

### test
minmax_test = scaler.transform(test_x[scale_list])

임시_df= pd.DataFrame(minmax_test, columns=scale_list)
minmax_test_x = pd.concat([임시_df,test_수명주기],axis=1)


In [18]:
mm_train = pd.concat([minmax_train_x, train_y],axis=1)
mm_test = pd.concat([minmax_test_x, test_y],axis=1)

minmax_df = pd.concat([mm_train,mm_test],axis=0).reset_index(drop=True)
minmax_df = pd.concat([id_col,minmax_df],axis=1)
minmax_df.to_csv('./datasets/스케일링완료/윈저_minmax.csv',encoding='utf-8-sig')

## 다시 csv로 저장
## 근데 이거 돌려보니까 엄청크면 1 엄청작으면 0으로 과대해석하는 경향이 보임
## 하드코딩으로 (x-min)/(max-min)으로 하거나
## log취하고 하는게 맞아보임

## MAXABSscaler

In [19]:
scaler = MaxAbsScaler()
### train
maxabs_train = scaler.fit_transform(train_x[scale_list])

임시_df= pd.DataFrame(maxabs_train, columns=scale_list)
maxabs_train_x = pd.concat([임시_df,train_수명주기],axis=1)

### test
maxabs_test = scaler.transform(test_x[scale_list])

임시_df= pd.DataFrame(maxabs_test, columns=scale_list)
maxabs_test_x = pd.concat([임시_df,test_수명주기],axis=1)


In [20]:
ma_train = pd.concat([maxabs_train_x, train_y],axis=1)
ma_test = pd.concat([maxabs_test_x, test_y],axis=1)

maxabs_df = pd.concat([ma_train,ma_test],axis=0).reset_index(drop=True)
maxabs_df = pd.concat([id_col,maxabs_df],axis=1)
maxabs_df.to_csv('./datasets/스케일링완료/윈저_maxabs.csv',encoding='utf-8-sig')


## robustScaler

In [21]:
scaler = RobustScaler()
### train
robust_train = scaler.fit_transform(train_x[scale_list])

임시_df= pd.DataFrame(robust_train, columns=scale_list)
robust_train_x = pd.concat([임시_df,train_수명주기],axis=1)

### test
robust_test = scaler.transform(test_x[scale_list])

임시_df= pd.DataFrame(robust_test, columns=scale_list)
robust_test_x = pd.concat([임시_df,test_수명주기],axis=1)


In [22]:
robust_train = pd.concat([robust_train_x, train_y],axis=1)
robust_test = pd.concat([robust_test_x, test_y],axis=1)

robust_df = pd.concat([robust_train,robust_test],axis=0).reset_index(drop=True)
robust_df = pd.concat([id_col,robust_df],axis=1)
robust_df.to_csv('./datasets/스케일링완료/윈저_robust.csv',encoding='utf-8-sig')


# isol

In [23]:
df = pd.read_csv('datasets\이상치제거후vif공선성제거\isol_vif.csv',index_col=0)
df.columns

Index(['target', '거래소코드', '상장일', '수명주기', '통계청 한국표준산업분류 10차(대분류)', '회계년도',
       '회사명', '대주주지분변화분', '외국인지분분변화', '자산', '당기순이익(손실)', '비유동자산증가율', '유동자산증가율',
       '자기자본증가율', '매출액증가율', '정상영업이익증가율', '순이익증가율', '총포괄이익증가율', '매출액순이익률',
       '자기자본순이익률', '광고선전비대매출액비율', '비유동비율', '당좌비율', '현금비율', '부채비율',
       '이자보상배율(이자비용)', '유보액대납입자본배율', '투자집중도', '1인년간평균급여(직원)', '토빈Q', 'WW지수',
       'RDS', '산업평균_총자산증가율차분', '보통주식비율', '외국인_주식분포비율', '업력'],
      dtype='object')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8916 entries, 0 to 9904
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   target                 8916 non-null   int64  
 1   거래소코드                  8916 non-null   int64  
 2   상장일                    8916 non-null   object 
 3   수명주기                   8916 non-null   object 
 4   통계청 한국표준산업분류 10차(대분류)  8916 non-null   object 
 5   회계년도                   8916 non-null   int64  
 6   회사명                    8916 non-null   object 
 7   대주주지분변화분               8916 non-null   float64
 8   외국인지분분변화               8916 non-null   float64
 9   자산                     8916 non-null   float64
 10  당기순이익(손실)              8916 non-null   float64
 11  비유동자산증가율               8916 non-null   float64
 12  유동자산증가율                8916 non-null   float64
 13  자기자본증가율                8916 non-null   float64
 14  매출액증가율                 8916 non-null   float64
 15  정상영업

In [25]:
## 11~12년도는 y taget 설정조건으로 인하여 제거

bef_df = df[~(df['회계년도'].isin([2011,2012]))].reset_index(drop=True)

In [26]:
id_col = bef_df[['회사명','회계년도','상장일','통계청 한국표준산업분류 10차(대분류)','거래소코드']]
bef_df.drop(['회사명','거래소코드','상장일','통계청 한국표준산업분류 10차(대분류)'],axis=1,inplace=True)

In [27]:
### 성장주기 변수 onehot인코딩 get_dummy()
dummy = pd.get_dummies(bef_df['수명주기'])
bef_df.drop(['수명주기'],axis=1,inplace=True)
bef_df = pd.concat([bef_df,dummy],axis=1)

In [28]:
### 회계년도 기준 split

train = bef_df[bef_df['회계년도']<=2019].reset_index(drop=True)
test = bef_df[bef_df['회계년도']>=2020].reset_index(drop=True)


train_y = train[['target']]
train_x = train[[ '대주주지분변화분', '외국인지분분변화', '자산', '당기순이익(손실)', '비유동자산증가율',
       '유동자산증가율', '자기자본증가율', '매출액증가율', '정상영업이익증가율', '순이익증가율', '총포괄이익증가율',
       '매출액순이익률', '자기자본순이익률', '광고선전비대매출액비율', '비유동비율', '당좌비율', '현금비율', '부채비율',
       '이자보상배율(이자비용)', '유보액대납입자본배율', '투자집중도', '1인년간평균급여(직원)', '토빈Q', 'WW지수',
       'RDS', '산업평균_총자산증가율차분', '보통주식비율', '외국인_주식분포비율', '업력', '도입기', '성숙기',
       '성장기', '쇠퇴기']]






test_y = test[['target']]
test_x = test[[ '대주주지분변화분', '외국인지분분변화', '자산', '당기순이익(손실)', '비유동자산증가율',
       '유동자산증가율', '자기자본증가율', '매출액증가율', '정상영업이익증가율', '순이익증가율', '총포괄이익증가율',
       '매출액순이익률', '자기자본순이익률', '광고선전비대매출액비율', '비유동비율', '당좌비율', '현금비율', '부채비율',
       '이자보상배율(이자비용)', '유보액대납입자본배율', '투자집중도', '1인년간평균급여(직원)', '토빈Q', 'WW지수',
       'RDS', '산업평균_총자산증가율차분', '보통주식비율', '외국인_주식분포비율', '업력', '도입기', '성숙기',
       '성장기', '쇠퇴기']]

scale_list = [ '대주주지분변화분', '외국인지분분변화', '자산', '당기순이익(손실)', '비유동자산증가율',
       '유동자산증가율', '자기자본증가율', '매출액증가율', '정상영업이익증가율', '순이익증가율', '총포괄이익증가율',
       '매출액순이익률', '자기자본순이익률', '광고선전비대매출액비율', '비유동비율', '당좌비율', '현금비율', '부채비율',
       '이자보상배율(이자비용)', '유보액대납입자본배율', '투자집중도', '1인년간평균급여(직원)', '토빈Q', 'WW지수',
       'RDS', '산업평균_총자산증가율차분', '보통주식비율', '외국인_주식분포비율', '업력']

# 범주형 변수 분리

In [29]:
train_수명주기 = train_x[[ '도입기','성숙기', '성장기', '쇠퇴기']] 
test_수명주기 = test_x[[ '도입기','성숙기', '성장기', '쇠퇴기']]
# id_col
# 3개의 데이터프레임을 마지막에 다시 합쳐준다

## Standarscaler

In [30]:
scaler = StandardScaler()
### train
standar_train = scaler.fit_transform(train_x[scale_list])

임시_df= pd.DataFrame(standar_train, columns=scale_list)
standar_train_x = pd.concat([임시_df,train_수명주기],axis=1)

### test
standar_test = scaler.transform(test_x[scale_list])

임시_df= pd.DataFrame(standar_test, columns=scale_list)
standar_test_x = pd.concat([임시_df,test_수명주기],axis=1)


In [31]:
stand_train = pd.concat([standar_train_x, train_y],axis=1)
stand_test = pd.concat([standar_test_x, test_y],axis=1)

standar_df = pd.concat([stand_train,stand_test],axis=0).reset_index(drop=True)
standar_df = pd.concat([id_col,standar_df],axis=1)
standar_df.to_csv('./datasets/스케일링완료/isol_standar.csv',encoding='utf-8-sig')

## 다시 csv로 저장

# MinMAxscaler

In [32]:
scaler = MinMaxScaler()
### train
minmax_train = scaler.fit_transform(train_x[scale_list])

임시_df= pd.DataFrame(minmax_train, columns=scale_list)
minmax_train_x = pd.concat([임시_df,train_수명주기],axis=1)

### test
minmax_test = scaler.transform(test_x[scale_list])

임시_df= pd.DataFrame(minmax_test, columns=scale_list)
minmax_test_x = pd.concat([임시_df,test_수명주기],axis=1)


In [33]:
mm_train = pd.concat([minmax_train_x, train_y],axis=1)
mm_test = pd.concat([minmax_test_x, test_y],axis=1)

minmax_df = pd.concat([mm_train,mm_test],axis=0).reset_index(drop=True)
minmax_df = pd.concat([id_col,minmax_df],axis=1)
minmax_df.to_csv('./datasets/스케일링완료/isol_minmax.csv',encoding='utf-8-sig')

## 다시 csv로 저장
## 근데 이거 돌려보니까 엄청크면 1 엄청작으면 0으로 과대해석하는 경향이 보임
## 하드코딩으로 (x-min)/(max-min)으로 하거나
## log취하고 하는게 맞아보임

# maxabsscaler


In [34]:
scaler = MaxAbsScaler()
### train
maxabs_train = scaler.fit_transform(train_x[scale_list])

임시_df= pd.DataFrame(maxabs_train, columns=scale_list)
maxabs_train_x = pd.concat([임시_df,train_수명주기],axis=1)

### test
maxabs_test = scaler.transform(test_x[scale_list])

임시_df= pd.DataFrame(maxabs_test, columns=scale_list)
maxabs_test_x = pd.concat([임시_df,test_수명주기],axis=1)


In [35]:
ma_train = pd.concat([maxabs_train_x, train_y],axis=1)
ma_test = pd.concat([maxabs_test_x, test_y],axis=1)

maxabs_df = pd.concat([ma_train,ma_test],axis=0).reset_index(drop=True)
maxabs_df = pd.concat([id_col,maxabs_df],axis=1)
maxabs_df.to_csv('./datasets/스케일링완료/isol_maxabs.csv',encoding='utf-8-sig')


# robust

In [36]:
scaler = RobustScaler()
### train
robust_train = scaler.fit_transform(train_x[scale_list])

임시_df= pd.DataFrame(robust_train, columns=scale_list)
robust_train_x = pd.concat([임시_df,train_수명주기],axis=1)

### test
robust_test = scaler.transform(test_x[scale_list])

임시_df= pd.DataFrame(robust_test, columns=scale_list)
robust_test_x = pd.concat([임시_df,test_수명주기],axis=1)


In [37]:
robust_train = pd.concat([robust_train_x, train_y],axis=1)
robust_test = pd.concat([robust_test_x, test_y],axis=1)

robust_df = pd.concat([robust_train,robust_test],axis=0).reset_index(drop=True)
robust_df = pd.concat([id_col,robust_df],axis=1)
robust_df.to_csv('./datasets/스케일링완료/isol_robust.csv',encoding='utf-8-sig')
